## Aprendizado Supervisionado
### prof. Duncan
Prática com KNN e Breast Cancer

Experimentos com Treino, Validação e Teste

In [ ]:
#pacote para profiling de datasets
!pip install ydata_profiling
# pacotes básicos
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport

# pacotes do sklearn para acesso a datasets, preparação, modelagem e avaliação
from sklearn import datasets
# pacote pipeline para combinar preparação e modelagem
from sklearn.pipeline import Pipeline, make_pipeline
# pacote para separação entre treino e teste
from sklearn.model_selection import train_test_split
# pacotes para validação cruzada
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
# arsenal de preparação
from sklearn.preprocessing import MinMaxScaler # rescala em min-max
from sklearn.preprocessing import StandardScaler # padroniza features removendo média e
#     escalando para variância unitária. Também chamado de z-score
#
# algoritmo de aprendizado
from sklearn.neighbors import KNeighborsClassifier
#
# medidas de desempenho
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
#
# tabulação da matriz de confusão
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.metrics import RocCurveDisplay

#pacotes para apoio a leitura e gravação de datasets
from pathlib import Path
import csv
import pprint

In [ ]:
# carga de dados
bc = datasets.load_breast_cancer(as_frame=True)
print(bc.DESCR)

In [ ]:
# separação em features e target e profiling do dataset
X = bc.data
y = bc.target
Xy = bc.frame
relatorio = ProfileReport(Xy, title="Breast Cancer data set")
relatorio.to_notebook_iframe()


In [ ]:
Xy

In [ ]:
# separação em treino e teste, e X e y
# importante fazer ANTES do pipeline para evitar data leakage
# o treino, como vai ser usado GridSearchCV, vai ter treino e validação

treino_X, teste_X, treino_y, teste_y = train_test_split(X, y, random_state=46,test_size=0.2, stratify=y)


In [ ]:
# configurações para experimentos

n_vizinhos = [1, 3, 5, 7, 9, 11, 13]
#n_vizinhos = [1, 3, 5]

k_splits = 10


## Modelagem com validação cruzada e com busca exaustiva pelos parâmetros

In [ ]:
# indução do modelo para diferentes números de vizinhos com validação cruzada

parametros = { 'n_neighbors': n_vizinhos, 'weights':['uniform', 'distance']}
modelo = KNeighborsClassifier()
valcruz = GridSearchCV(modelo, parametros,cv=k_splits)
valcruz.fit(treino_X, treino_y)
resultados = pd.DataFrame(valcruz.cv_results_).sort_values(by='rank_test_score')
print('Melhor resultado - Número vizinhos:{}   Pesos:{}    Acurácia média:{:.3f}'.format(
    resultados['param_n_neighbors'].iloc[0],
    resultados['param_weights'].iloc[0],
    resultados['mean_test_score'].iloc[0]))


In [ ]:
resultados.head()

In [ ]:
valcruz.cv_results_['params']

# Métrica de avaliação: accuracy

## Experimento com reescala de valores e busca exaustiva nos parâmetros

In [ ]:
# protocolo experimental com preparação e modelagem
modelo = Pipeline(steps=[
    ('reescala', MinMaxScaler()),
#    ('reescala', StandardScaler()),
     ('modelagem', KNeighborsClassifier())
    ])
parametros = {'modelagem__n_neighbors':n_vizinhos, 'modelagem__weights':['uniform', 'distance']}
kfold = KFold(n_splits=k_splits, shuffle=True, random_state=0)
grade = GridSearchCV(modelo, param_grid=parametros, cv=kfold)
grade.fit(treino_X,treino_y)
print('Melhor k:{}  score:{:0.3f}'.format(grade.best_params_, grade.best_score_))

In [ ]:
grade

In [ ]:
resultados = pd.DataFrame(grade.cv_results_).sort_values(by='rank_test_score')
resultados.head()

### Captura dos melhores parâmetros para indução do modelo de aprendizado

In [ ]:
# captura da melhor configuração
n_neighbors_best = grade.best_params_['modelagem__n_neighbors']
n_weights_best = grade.best_params_['modelagem__weights']
escalonador = MinMaxScaler()
#escalonador = StandardScaler()
modelo = KNeighborsClassifier(n_neighbors=n_neighbors_best, weights=n_weights_best)
treino_X_escalonado = escalonador.fit_transform(treino_X, treino_y)
teste_X_escalonado = escalonador.transform(teste_X)
modelo.fit(treino_X_escalonado, treino_y)
teste_pred_y = modelo.predict(teste_X_escalonado)
acuracia = accuracy_score(teste_y, teste_pred_y)
resultado = confusion_matrix(teste_y, teste_pred_y)
confusion_matrix_display = ConfusionMatrixDisplay(resultado).plot()
fig = confusion_matrix_display.figure_
fig.set_figheight(3)
fig.set_figwidth(3)
print('k=', n_neighbors_best,'  Acuracia=', acuracia)

### Curva ROC com AUC

In [ ]:
RocCurveDisplay.from_estimator(modelo, teste_X_escalonado, teste_y)

# Métrica de avaliação: recall

## Experimento com reescala de valores e busca exaustiva nos parâmetros

In [ ]:
# protocolo experimental com preparação e modelagem
modelo = Pipeline(steps=[
    ('reescala', MinMaxScaler()),
#    ('reescala', StandardScaler()),
     ('modelagem', KNeighborsClassifier())
    ])
parametros = {'modelagem__n_neighbors':n_vizinhos, 'modelagem__weights':['uniform', 'distance']}
kfold = KFold(n_splits=k_splits, shuffle=True, random_state=0)
grade = GridSearchCV(modelo, param_grid=parametros, cv=kfold, scoring='recall')
grade.fit(treino_X,treino_y)
print('Melhor k:{}  score:{:0.3f}'.format(grade.best_params_, grade.best_score_))

In [ ]:
grade

In [ ]:
resultados = pd.DataFrame(grade.cv_results_).sort_values(by='rank_test_score')
resultados.head()

### Captura dos melhores parâmetros para indução do modelo de aprendizado

In [ ]:
# captura da melhor configuração
n_neighbors_best = grade.best_params_['modelagem__n_neighbors']
n_weights_best = grade.best_params_['modelagem__weights']
escalonador = MinMaxScaler()
#escalonador = StandardScaler()
modelo = KNeighborsClassifier(n_neighbors=n_neighbors_best, weights=n_weights_best)
treino_X_escalonado = escalonador.fit_transform(treino_X, treino_y)
teste_X_escalonado = escalonador.transform(teste_X)
modelo.fit(treino_X_escalonado, treino_y)
teste_pred_y = modelo.predict(teste_X_escalonado)
recall = recall_score(teste_y, teste_pred_y)
resultado = confusion_matrix(teste_y, teste_pred_y)
confusion_matrix_display = ConfusionMatrixDisplay(resultado).plot()
fig = confusion_matrix_display.figure_
fig.set_figheight(3)
fig.set_figwidth(3)
print('k=', n_neighbors_best,'  Revocação=', recall)

In [ ]:
RocCurveDisplay.from_estimator(modelo, teste_X_escalonado, teste_y)